### 1. One client to rule them all

The Connectome Annotation Versioning Engine framework consists of a number of different services, each with a specific set of tasks that it can perform through REST endpoints. This module is designed to ease programmatic interaction with all of the various endpoints. Going forward, we also will be increasingly using authentication tokens for programmatic access to most if not all of the services. In order to collect a given server, datastack name, and user token together into a coherent package that can be used on multiple endpoints, we will use a CAVEclient that can build appropriately configured clients for each of the specific services.

The following examples cover each of the client subtypes that are associated with a single service. The ImageryClient, which is a more complex collection of tools, will be covered elsewhere.

#### Setting up your credentials

Most services require you to authenticate yourself. In order to use authenticated services in the CAVEclient, you need to get a token for yourself and save it to your hard drive. This token should be treated as a personal secret, like a password.

The easiest way to get your token is to use the client in its so-called "global" mode, without reference to any particular dataset. Once your client is initialized, you can get instructions for how to get a new token by running `client.auth.get_new_token()`. If your server is not for the MICrONs project, you may need to set a server address when initializing the CAVEclient.

Note: If you have already set up Graphene authetication in CloudVolume, the same token applies. The CAVEclient will read from and write to the same default file as CloudVolume, so you probably need to do nothing to use it. If you need multiple tokens for different projects, please read the documentation.


In [1]:
from caveclient import CAVEclient

client = CAVEclient()
client.auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


Follow the instructions that are printed out. If you cannot log in in step 2, contact the project administrators. Copy the token (only the long alphanumeric part!) and save it to your drive by running the command specified in step 3a after replacing the PASTE_YOUR_TOKEN_HERE bit with the value you got after login. Once this is done, you shouldn't need to do it again for a long while. Note that every time you do the refresh_token step, you invalidate any previous token. That means that if you want to use the CAVEclient on multiple computers, you need to copy the same token to each device instead of going through these steps multiple times.

In [2]:
client.auth.save_token(token="792ec77100af08528fd6a9d9ece61877")
client.auth.token = "792ec77100af08528fd6a9d9ece61877"

#### Initializing a CAVEclient

Most services require the use of a specific datastack. Once you have set up credentials on your computer. we can specify a datastack.

In [3]:
from caveclient import CAVEclient

datastack_name = 'pinky_sandbox'
client = CAVEclient(datastack_name)

Just to confirm that this works, let's see if we can get the EM image source from the InfoService. If you get a reasonable looking path, everything is okay.

In [4]:
print(f"The image source is: {client.info.image_source()}")

The image source is: precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/pinky/em


If you don't know what datastacks exist, you can start a client with or without a datastack name (like we did when we set the auth token) and run `client.info.get_datastacks()`

### 1.1 Query table

In [30]:
#df=client.materialize.query_table('https://minnie.microns-daf.com/annotation/views/aligned_volume/pinky100/table/synapses', limit=10)
pinky_sandbox = client.info.get_datastack_info('pinky_sandbox')
print(pinky_sandbox)
df=client.materialize.query_table(table="synapses", limit=10) 
df

{'synapse_table': 'synapses', 'cell_identification_table': None, 'local_server': 'https://minnie.microns-daf.com', 'segmentation_source': 'graphene://https://minnie.microns-daf.com/segmentation/table/pinky_nf_v2', 'analysis_database': None, 'viewer_resolution_x': 4.0, 'viewer_resolution_y': 4.0, 'viewer_resolution_z': 40.0, 'proofreading_status_table': None, 'aligned_volume': {'id': 3, 'display_name': 'Pinky', 'description': 'The MiCrONS phase 2 100um dataset from layer 2/3 of p32 mouse visual cortex.', 'image_source': 'precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/pinky/em', 'name': 'pinky100'}, 'description': 'This is a sandbox segmentation for pinky that runs on the same chunkedgraph version as minnie, but has had very little proofreading done, and is used for testing purposes and not science.', 'proofreading_review_table': None, 'soma_table': 'nuclei', 'viewer_site': 'https://neuromancer-seung-import.appspot.com/'}


201 - "Limited query to 10 rows, Table Owner Notice on synapses:


,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,0,2023-05-19 20:34:10.708584+00:00,NaN,t,1103,74382099528296136,648518346351966248,74382099528315165,648518346352385726,"[70050, 65534, 1015]","[70104, 65602, 1015]","[70196, 65474, 1015]"
1,1,2023-05-19 20:34:10.708584+00:00,NaN,t,62,75789955246878111,648518346351307508,75789955246883494,648518346351029486,"[91472, 73196, 645]","[91396, 73216, 645]","[91458, 73202, 645]"
2,2,2023-05-19 20:34:10.708584+00:00,NaN,t,62,73465519281597001,648518346353908527,73465519281616832,648518346351681764,"[57250, 39028, 1202]","[57312, 38966, 1203]","[57286, 38988, 1202]"
3,3,2023-05-19 20:34:10.708584+00:00,NaN,t,955,76351531146295535,648518346355903299,76351531146274880,648518346354030502,"[99538, 52184, 1276]","[99550, 52098, 1276]","[99613, 52140, 1269]"
4,4,2023-05-19 20:34:10.708584+00:00,NaN,t,718,77478049595342861,648518346354779254,77478049595363078,648518346352496854,"[115462, 61350, 1335]","[115468, 61296, 1335]","[115467, 61303, 1339]"
5,5,2023-05-19 20:34:10.708584+00:00,NaN,t,62,74522425236665046,648518346353694398,74522425236661536,648518346351917502,"[72464, 60164, 2059]","[72384, 60180, 2059]","[72434, 60186, 2059]"
6,6,2023-05-19 20:34:10.708584+00:00,NaN,t,1469,76280337700949909,648518346352356187,76280337700955893,648518346354130198,"[97772, 40340, 1066]","[97768, 40430, 1066]","[97824, 40388, 1066]"
7,7,2023-05-19 20:34:10.708584+00:00,NaN,t,62,74382580832756013,648518346352259774,74382580832760822,648518346349540258,"[70180, 72650, 1448]","[70148, 72556, 1448]","[70176, 72598, 1448]"
8,8,2023-05-19 20:34:10.708584+00:00,NaN,t,62,76844043501807823,648518346353145453,76844043501808008,648518346355652026,"[106814, 51086, 1019]","[106726, 51108, 1020]","[106766, 51084, 1020]"
9,9,2023-05-19 20:34:10.708584+00:00,NaN,t,202,77618718632685193,648518346354027962,77618718632681702,648518346353220028,"[117858, 61074, 1863]","[117898, 61006, 1864]","[117890, 61028, 1862]"


### 2. Authentication Service

The AuthClient handles storing and loading your token or tokens and inserting it into requests in other clients. We can access the auth client from `client.auth`. Once you have saved a token, you probably won't interact with this client very often, however it makes it convenient for saving new tokens the first time and has some other handy features. For example, to check what your token is (for example to set up a second computer).

In [5]:
auth = client.auth
print(f"My current token is: {auth.token}")

My current token is: 792ec77100af08528fd6a9d9ece61877


#### Loading saved tokens
The credentials save by default in `~/.cloudvolume/secrets/cave-secret.json`. You can try opening that file to see what we just created.

If we had wanted to use a different file or a different json key, we could have specified that in `auth.save_token`.

Because we used the default values, this token is used automatically when we intialize a new CAVEclient. If we wanted to use a different token file, token key, or even directly specify a token we could also do so here. Of course, a bad token will cause an unauthorized error.

In [ ]:
client = CAVEclient(datastack_name)
print(f"Now my basic token is: {client.auth.token}")

client_direct = CAVEclient(datastack_name, auth_token='another_fake_token_678')
print(f"A directly specified token is: {client_direct.auth.token}")

If you use a CAVEclient, the AuthClient and its token will be automatically applied to any other services without further use. 

### 3. Info Service
A datastack has a number of complex paths to various data sources that together comprise a datastack. Rather than hardcode these paths, the InfoService allows one to query the location of each data source. This is also convenient in case data sources change.

An InfoClient is accessed at `client.info`.

In [6]:
client.info

In [7]:
client = CAVEclient(datastack_name)
print(f"This is an info client for {client.info.datastack_name} on {client.info.server_address}")

This is an info client for pinky_sandbox on https://global.daf-apis.com


#### Accessing datastack information
All of the information accessible for the datastack can be seen as a dict using `get_datastack_info()`.

In [ ]:
client.info.get_datastack_info()

Individual entries can be found as well. Use tab autocomplete to see the various possibilities.

In [ ]:
client.info.local_server()

### 4. JSON Service

We store the JSON description of a Neuroglancer state in a simple database at the JSON Service. This is a convenient way to build states to distribute to people, or pull states to parse work by individuals. The JSON Client is at `client.state`. Note that the state service will work with or without a datastack set.

In [ ]:
client.state

#### Retrieving a state

JSON states are found simply by their ID, which you get when uploading a state. You can download a state with `get_state_json`.

In [ ]:
example_id = 5762925562167296
example_state = client.state.get_state_json(example_id)
example_state['layers'][0]

#### Uploading a state
You can also upload states with `upload_state_json`. If you do this, the state id is returned by the function. Note that there is no easy way to query what you uploaded later, so be VERY CAREFUL with this state id if you wish to see it again.

*Note: If you are working with a Neuroglancer Viewer object or similar, in order to upload, use viewer.state.to_json() to generate this representation.*

In [ ]:
example_state['layers'][0]['name'] = 'example_name'
new_id = client.state.upload_state_json(example_state)

In [ ]:
test_state = client.state.get_state_json(new_id)
test_state['layers'][0]['name']

#### Generating Neuroglancer URLs

Neuroglancer can automatically look up a JSON state based on its ID if you pass the URL to it correctly. The function `build_neuroglancer_url` helps format these correctly. Note that you need to specify the base URL for the Neuroglancer deployment you wish to use.

In [ ]:
url = client.state.build_neuroglancer_url(state_id=new_id, ngl_url='https://neuromancer-seung-import.appspot.com')
print(url)

### 5. ChunkedGraph

The ChunkedGraph client allows one to interact with the ChunkedGraph, which stores and updates the supervoxel agglomeration graph. This is most often useful for looking up an object root id of a supervoxel or looking up supervoxels belonging to a root id. The ChunkedGraph client is at `client.chunkedgraph`.

#### Look up a supervoxel
Usually in Neuroglancer, one never notices supervoxel ids, but they are important for programmatic work. In order to look up the root id for a location in space, one needs to use the supervoxel segmentation to get the associated supervoxel id. The ChunkedGraph client makes this easy using the `get_root_ids` method.

In [ ]:
from caveclient import CAVEclient

datastack_name = 'minnie65_phase3_v0'
client = CAVEclient(datastack_name)

In [ ]:
sv_id = 109362238070465629
client.chunkedgraph.get_root_id(supervoxel_id=sv_id)

However, as proofreading occurs, the root id that a supervoxel belongs to can change. By default, this function returns the current state, however one can also provide a UTC timestamp to get the root id at a particular moment in history. This can be useful for reproducible analysis.

In [ ]:
import datetime

date_3_days_ago = datetime.datetime.now() - datetime.timedelta(days=3)

# I looked up the UTC POSIX timestamp from a day in early 2019. 
#timestamp = datetime.datetime.utcfromtimestamp(1546595253)

client.chunkedgraph.get_root_id(supervoxel_id=sv_id, timestamp=date_3_days_ago)

#### Getting supervoxels for a root id

A root id is associated with a particular agglomeration of supervoxels, which can be found with the `get_leaves` method. A new root id is generated for every new change in the chunkedgraph, so time stamps do not apply.

In [ ]:
root_id = 864691134988869442
client.chunkedgraph.get_leaves(root_id)

### 7. AnnotationEngine

The AnnotationClient is used to interact with the AnnotationEngine service to create tables from existing schema, upload new data, and download existing annotations. Note that annotations in the AnnotationEngine are not linked to any particular segmentation, and thus do not include any root ids. An annotation client is accessed with `client.annotation`.

#### Get existing tables

A list of the existing tables for the datastack can be found at with `get_tables`.

In [ ]:
all_tables = client.annotation.get_tables()
all_tables

Each table has three main properties that can be useful to know:
* `table_name` : The table name, used to refer to it when uploading or downloading annotations. This is also passed through to the table in the Materialized database.
* `schema_name` : The name of the table's schema from EMAnnotationSchemas (see below).
* `max_annotation_id` : An upper limit on the number of annotations already contained in the table.

#### Downloading annotations

You can download the JSON representation of a data point through the `get_annotation` method. This can be useful if you need to look up information on unmaterialized data, or to see what a properly templated annotation looks like.

In [ ]:
table_name = all_tables[0]
annotation_id = 1
client.annotation.get_annotation(annotation_ids=1, table_name=table_name)

#### Create a new table

One can create a new table with a specified schema with the `create_table` method:

```
client.annotation.create_table(table_name='test_table',
                               schema_name='microns_func_coreg')
```

Now, new data can be generated as a dict or list of dicts following the schema and uploaded with `post_annotation`.
For example, a `microns_func_coreg` point needs to have:
    * `type` set to `microns_func_coreg`
    * `pt` set to a dict with `position` as a key and the xyz location as a value.
    * `func_id` set to an integer.
    
The following will create a new annotation and upload it to the service: 
```
new_data = {'type': 'microns_func_coreg',
            'pt': {'position': [1,2,3]},
            'func_id': 0}
            
client.annotation.post_annotation(table_name='test_table', data=[new_data])
```

### 7. EMAnnotationSchemas

The EMAnnotationSchemas client lets one look up the available schemas and how they are defined. This is mostly used for programmatic interactions between services, but can be useful when looking up schema definitions for new tables.

#### Get the list of schema 
One can get the list of all available schema with the `schema` method. Currently, new schema have to be generated on the server side, although we aim to have a generic set available to use.

In [ ]:
client.schema.schema()

#### View a specific schema

The details of each schema can be viewed with the `schema_definition` method, formatted as per JSONSchema. 

In [ ]:
example_schema = client.schema.schema_definition('microns_func_coreg')
example_schema

This is mostly useful for programmatic interaction between services at the moment, but can also be used to inspect the expected form of an annotation by digging into the format.

In [ ]:
example_schema['definitions']['FunctionalCoregistration']